In [1]:
import pandas as pd
import numpy as np

In [2]:
### Load median revenue by code INSEE (from INSEE)
df_median_rev = pd.read_excel('./Vertbaudet/France/INSEE_data/IRIS_com_and_arr_2018.xlsx')

In [1]:
### Load correspondance between ZIP code and INSEE code (from Data.gouv)
df_corresp = pd.read_excel('./Vertbaudet/France/INSEE_data/rev_iris_by_zip.xlsx', sheet_name = 'correspondance-code-insee-code-', usecols = ['Code INSEE', 'Code Postal'])

NameError: name 'pd' is not defined

In [4]:
### Take only numeric CODGEO (thus excluding Corsica)
df_median_rev = df_median_rev[df_median_rev.CODGEO.str.isnumeric()]

### Convert codes to int for merge
df_median_rev['CODGEO'] = df_median_rev['CODGEO'].astype(int)
df_corresp['Code INSEE'] = df_corresp['Code INSEE'].astype(int)

### Merge
df = df_median_rev.merge(df_corresp, how = 'left', left_on = 'CODGEO', right_on = 'Code INSEE')

### Some Codepost are of the form 95000/95100, we choose then to only keep the first postcode
def clean_codpost(x):
    if len(str(x)) == 11:
        return str(x)[:5]
    else :
        return x
    
df['Code Postal'] = df['Code Postal'].apply(clean_codpost)

### If several CODGEO are under one Code Postal then take the mean revenue of the Code Postal
df = df.groupby('Code Postal').agg({'MED18': 'mean'})

In [31]:
df.to_excel('./Vertbaudet/France/INSEE_data/Median_rev_by_zip.xlsx')